### Aqui extraímos os dados geográfico separados por setor censitário

In [13]:
import ftplib
import re
import json
import zipfile
import sqlite3
import geopandas
import itertools
import operator
import sys
import shapely.wkt
import fiona


from tqdm import tqdm_notebook
from path import Path

data_path = Path().abspath().parent.joinpath('data')
data_path

sys.path.insert(0, data_path.parent)

import libcenso
import imp
imp.reload(libcenso)

<module 'libcenso' from '/home/diogenes/PycharmProjects/libminer/libcenso.py'>

#### Primeiro, obter uma lista de todos os zips com os dados geográficos

In [1]:
host = 'geoftp.ibge.gov.br'
url = ('organizacao_do_territorio/malhas_territoriais/'
       'malhas_de_setores_censitarios__divisoes_intramunicipais/'
       'censo_2010/setores_censitarios_shp')

In [2]:
print('ftp://' + host + '/' + url)

ftp://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2010/setores_censitarios_shp


In [18]:
def chamar_com_cache(path, func, *args, **kwargs):
    """
    Tenta obter o valor de retorno da função de um cache no
    arquivo indicado em path.
    
    Caso não consiga, chama a função func(*args, **kwargs), 
    salvando no arquivo de cache.
    """
    try:
        with open(path) as fp:
            result = json.load(fp)
        print("Carregou do cache")
    except (IOError, ValueError):
        result = func(*args, **kwargs)
        
        with open(path, 'w') as fp:
            json.dump(result, fp, indent=2)
        print("Salvou no cache")
    finally:
        return result

In [19]:
def obter_urls_zips_estados(host, url):
    """
    Retorna um dicionário {estado: (url, codigo)} com os zips dos
    shapefiles dos dados geográficos.
    """
    zips_estados = {}
    ftp = ftplib.FTP(host)
    ftp.login()

    for estado_dir in ftp.nlst(url):
        basename = Path(estado_dir).basename()
        
        if re.match('''[a-z]{2}$''', basename):
            estado = basename.upper()
            for fzip in ftp.nlst(estado_dir):
                if 'censitario' in Path(fzip).basename():
                    zips_estados[estado] = fzip
    return zips_estados

In [20]:
zips_estados = chamar_com_cache(data_path.joinpath('zips_estados.json'),
                                obter_urls_zips_estados, host, url)

Carregou do cache


#### Agora, baixar os dados dos setores censitários

In [21]:
setores_path = data_path.joinpath('setores-censitarios').mkdir_p()
setores_path

Path('/home/diogenes/PycharmProjects/libminer/data/setores-censitarios')

In [22]:
ftp = ftplib.FTP(host)
ftp.login()

for estado, path in tqdm_notebook(zips_estados.items(), desc='Total'):
    zip_fname = setores_path.joinpath(estado.lower() + '_setores_censitarios.zip')
    
    # Já existe
    if zip_fname.isfile():
        continue
    
    tamanho = ftp.size(path)
    comando = 'RETR ' + path
    pbar = tqdm_notebook(total=tamanho, desc=estado)
    fp = open(zip_fname, 'wb')
    
    def write_to_file(block):
        fp.write(block)
        pbar.update(len(block))
    
    ftp.retrbinary(comando, write_to_file)
    fp.close()
    pbar.close()

#### Extrair cada um para uma pasta correspondente

In [23]:
setores_path = data_path.joinpath('setores-censitarios').mkdir_p()
setores_path

Path('/home/diogenes/PycharmProjects/libminer/data/setores-censitarios')

In [24]:
zips_fnames = list(setores_path.glob('*_setores_censitarios.zip'))

for zip_fname in tqdm_notebook(zips_fnames, desc='Total'):
    estado = Path(zip_fname).basename()[:2].upper()
    path, _ = Path(zip_fname).splitext()
    path.mkdir_p()
    
    with zipfile.ZipFile(zip_fname) as fp:
        fp.extractall(path)

# Popular as tabelas

#### Tabela de estados

In [5]:
setores_path = data_path.joinpath('setores-censitarios').mkdir_p()
setores_path

Path('/home/diogenes/PycharmProjects/libminer/data/setores-censitarios')

In [7]:
with open(data_path.joinpath('estados_nomes.json')) as fp:
    estados_nomes = json.load(fp)

In [8]:
setores_pastas = list(setores_path.glob('*_setores_censitarios'))
shapes_estado = {}

for pasta in setores_pastas:
    estado = pasta.basename()[:2].upper()
    
    shp_file = list(pasta.glob('*.shp'))[0]
    codigo = int(shp_file.basename()[:2])
    
    shapes_estado[estado] = codigo, shp_file

estados_campos = [(i + 1, estado, codigo, estados_nomes[estado].upper())
                  for i, (estado, (codigo, shp_file))
                  in enumerate(shapes_estado.items())]

In [31]:
con = sqlite3.connect(data_path.joinpath('CENSO2010.sqlite'))
con.row_factory = sqlite3.Row

In [32]:
query = 'INSERT INTO estados (id, sigla_estado, codigo_estado, nome_estado) VALUES (?, ?, ?, ?)'
con.executemany(query, estados_campos)
con.commit()
con.close()

#### Tabela de municípios

In [36]:
con.close()

In [47]:
con = sqlite3.connect(data_path.joinpath('CENSO2010.sqlite'))
query = 'INSERT INTO municipios (id_estado, codigo_municipio, nome_municipio) VALUES (?, ?, ?)'

In [48]:
resultados = list(pesquisar_geo(con, 'sigla_estado', 'id_estado'))

for info in tqdm_notebook(resultados, desc='Total'):
    sigla, id_estado = info['sigla_estado'], info['id_estado']
    df = geopandas.read_file(shapes_estado[sigla][1])
    
    municipios_unique = df.drop_duplicates('CD_GEOCODM')
    
    codigos = list(map(str, municipios_unique['CD_GEOCODM']))
    nomes = list(map(str, municipios_unique['NM_MUNICIP']))
    
    con.executemany(query, zip(
        itertools.repeat(id_estado),
        codigos,
        nomes
    ))

In [42]:
list(pesquisar_geo(con, 'nome_municipio', sigla_estado='DF'))

[{'nome_municipio': 'BRASÍLIA'}]

In [54]:
con.commit()
con.close()

#### Tabela de distritos

In [45]:
con = sqlite3.connect(data_path.joinpath('CENSO2010.sqlite'))
query_distrito = ('INSERT INTO distritos (id_estado, id_municipio, codigo_distrito, nome_distrito) '
                  'VALUES (?, ?, ?, ?)')

In [53]:
resultados_estado = list(pesquisar_geo(con, 'sigla_estado', 'id_estado'))

for info in tqdm_notebook(resultados_estado, desc='Total'):
    sigla, id_estado = info['sigla_estado'], info['id_estado']
    df = geopandas.read_file(shapes_estado[sigla][1])
    
    resultados_municipio = list(pesquisar_geo(con, 'id_municipio', 'codigo_municipio',
                                             id_estado=id_estado))
    
    for mun in tqdm_notebook(resultados_municipio, desc=sigla):
        mun_df = df.loc[df['CD_GEOCODM'] == mun['codigo_municipio']]
        distritos_unique = mun_df.drop_duplicates('CD_GEOCODD')
        
        codigos = list(map(str, distritos_unique['CD_GEOCODD']))
        nomes = list(map(str, distritos_unique['NM_DISTRIT']))
        
        con.executemany(query_distrito, zip(
            itertools.repeat(id_estado),
            itertools.repeat(mun['id_municipio']),
            codigos,
            nomes
        ))
        

#### Tabela de subdistritos

In [12]:
con = sqlite3.connect(data_path.joinpath('CENSO2010.sqlite'))
query_subdistrito = ('INSERT INTO subdistritos '
                     '(id_estado, id_municipio, id_distrito, codigo_subdistrito, nome_subdistrito) '
                     'VALUES (?, ?, ?, ?, ?)')

In [1]:
import logging

In [2]:
?logging.getLogger

In [14]:
resultados_estado = list(pesquisar_geo(con, 'sigla_estado', 'id_estado'))

for info in tqdm_notebook(resultados_estado, desc='Total'):
    sigla, id_estado = info['sigla_estado'], info['id_estado']
    df = geopandas.read_file(shapes_estado[sigla][1])
    
    resultados_distrito = list(pesquisar_geo(con, 'id_municipio', 'id_distrito', 'codigo_distrito',
                                             id_estado=id_estado))
    
    for dis in tqdm_notebook(resultados_distrito, desc=sigla):
        dis_df = df.loc[df['CD_GEOCODD'] == dis['codigo_distrito']]
        subdistritos_unique = dis_df.drop_duplicates('CD_GEOCODS')
        
        codigos = list(map(str, subdistritos_unique['CD_GEOCODS']))
        nomes = list(map(str, subdistritos_unique['NM_SUBDIST']))
        
        con.executemany(query_subdistrito, zip(
            itertools.repeat(id_estado),
            itertools.repeat(dis['id_municipio']),
            itertools.repeat(dis['id_distrito']),
            codigos,
            nomes
        ))
        

In [15]:
con.commit()
con.close()

#### Tabela de setores censitários

In [35]:
con.close()
del con

In [36]:
con = sqlite3.connect(data_path.joinpath('CENSO2010.sqlite'))
query_setores = '''
    INSERT INTO
        setores
    (id_estado, id_municipio, id_distrito, id_subdistrito, codigo_setor,
        bairro, poligono, area, lng_min, lat_min, lng_max, lat_max, centroide)
    VALUES
        (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''

In [38]:
resultados_estado = list(pesquisar_geo(con, 'sigla_estado', 'id_estado'))

for info in tqdm_notebook(resultados_estado, desc='Total'):
    sigla, id_estado = info['sigla_estado'], info['id_estado']
        
    df = geopandas.read_file(shapes_estado[sigla][1])
    
    resultados_subdistrito = list(pesquisar_geo(con,
                                             'id_municipio', 'id_distrito', 'id_subdistrito', 'codigo_subdistrito',
                                             id_estado=id_estado))
    
    for subdis in tqdm_notebook(resultados_subdistrito, desc=sigla):
        subdis_df = df.loc[df['CD_GEOCODS'] == subdis['codigo_subdistrito']]
        codigos = list(map(str, subdis_df['CD_GEOCODI']))
        bairros = list(map(str, subdis_df['NM_BAIRRO']))
        poligonos = list(map(shapely.wkt.dumps, subdis_df.geometry))
        centroides = list(map(shapely.wkt.dumps, subdis_df.geometry.centroid))
        
        bounds_f = lambda p, i: list(map(float, p.bounds))[i]
        lng_min = list(map(lambda p: bounds_f(p, 0), subdis_df.geometry))
        lat_min = list(map(lambda p: bounds_f(p, 1), subdis_df.geometry))
        lng_max = list(map(lambda p: bounds_f(p, 2), subdis_df.geometry))
        lat_max = list(map(lambda p: bounds_f(p, 3), subdis_df.geometry))
        
        subdis_df.crs = fiona.crs.from_epsg(4674)
        converted = subdis_df.to_crs(epsg=4087)
        areas = list(map(float, converted.geometry.area))
        
        con.executemany(query_setores, zip(
            itertools.repeat(id_estado),
            itertools.repeat(subdis['id_municipio']),
            itertools.repeat(subdis['id_distrito']),
            itertools.repeat(subdis['id_subdistrito']),
            codigos,
            bairros,
            poligonos,
            areas,
            lng_min,
            lat_min,
            lng_max,
            lat_max,
            centroides
        ))

In [39]:
con.commit()
con.close()
del con

#### Testando som 1 2 3

In [36]:
con.close()

In [14]:
path = data_path.joinpath('CENSO2010.sqlite')
censo = libcenso.CensoBD()

In [15]:
censo.conectar(path)

In [22]:
list(censo.recuperar('nome_municipio', 'nome_estado', id_subdistrito=10909))

[{'nome_estado': 'RIO GRANDE DO SUL', 'nome_municipio': 'ALEGRETE'}]